In [15]:
from moviepy.video.io.ImageSequenceClip import ImageSequenceClip
import os
from tqdm import tqdm
imgs = sorted(os.listdir('/storage/valis_reg/rf_trained_pred/retinex'))
imgs = ['/storage/valis_reg/rf_trained_pred/retinex/'+img for img in imgs]


In [26]:
clip = ImageSequenceClip(imgs, durations=[60/2627]*2627)

In [13]:
import os
import cv2
from tqdm import tqdm
import numpy as np


original_frames = os.listdir('/storage/valis_reg/rf_trained_pred/original')
retinex_frames = os.listdir('/storage/valis_reg/rf_trained_pred/retinex')
final_masks = os.listdir('/storage/valis_reg/rf_trained_pred/processed_mask')
raw_masks = os.listdir('/storage/valis_reg/rf_trained_pred/raw_mask')
retinex_frames = sorted(retinex_frames, key=lambda x: int(x.split('_')[0]))
original_frames = sorted(original_frames, key=lambda x: int(x.split('_')[0]))
final_masks = sorted(final_masks, key=lambda x: int(x.split('_')[0]))
raw_masks = sorted(raw_masks, key=lambda x: int(x.split('_')[0]))

def pad_img_to_ref(img, ref):
    h, w = ref.shape[:2]
    img_h, img_w = img.shape[:2]
    pad_h = h - img_h
    pad_w = w - img_w
    top = pad_h//2
    bottom = pad_h - top
    left = pad_w//2
    right = pad_w - left
    return cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=[255, 255, 255])


def extract_img_from_ref_mask(img, ref_mask_v):
    blank= np.ones_like(img)*255
    # blank[ref_mask==1] = img[ref_mask==1]
    for i in range(3):
        blank[:,:,i][ref_mask_v[:,:,i] ==1 ] = img[:,:,i][ref_mask_v[:,:,i]==1]
    return blank


def prepare_stacked_mask(ref_mask):
    blank_ref = np.zeros((ref_mask.shape[0], ref_mask.shape[1], 3))
    blank_ref[:,:,0] = ref_mask
    blank_ref[:,:,1] = ref_mask
    blank_ref[:,:,2] = ref_mask
    # ref_mask_v = np.stack([ref_mask, ref_mask, ref_mask],axis=-1)
    return blank_ref

ref_frame = '/storage/valis_reg/rf_trained_pred/original/1606_original.jpg'
ref_mask = '/storage/valis_reg/rf_trained_pred/processed_mask/1606_mask.jpg'
ref_raw_mask = '/storage/valis_reg/rf_trained_pred/raw_mask/1606_raw_mask.jpg'
ref_img = cv2.imread(ref_frame)
ref_mask = cv2.imread(ref_mask, cv2.IMREAD_GRAYSCALE)
ref_raw_mask = cv2.imread(ref_raw_mask, cv2.IMREAD_GRAYSCALE)
ref_mask[ref_mask>0] = 1
ref_raw_mask[ref_raw_mask>0] = 1
ref_img = pad_img_to_ref(ref_img, ref_img)
ref_mask_v = prepare_stacked_mask(ref_mask)
ref_frame = extract_img_from_ref_mask(ref_img, ref_mask_v)

In [16]:
imgs = [pad_img_to_ref(cv2.imread(img_path), ref_frame) for img_path in tqdm(imgs)]

100%|██████████| 2627/2627 [00:30<00:00, 86.59it/s] 


In [28]:
for idx, img in enumerate(imgs):
    cv2.imwrite(f'padded_retinex/{idx}_retinex.jpg', img)